<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/notebooks/FinanceBench_chromaDB_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install langchain chromadb sentence_transformers unstructured jq numpy==1.24.4 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install langchain_community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import chromadb
from langchain.retrievers import ParentDocumentRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import JSONLoader
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore
import os

In [6]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Define the metadata extraction function.
def metadata_function(record, metadata) -> dict:

    metadata["cik"] = record.get("cik")
    metadata["company"] = record.get("company")
    metadata["filing_type"] = record.get("filing_type")
    metadata["filing_date"] = record.get("filing_date")
    metadata["period_of_report"] = record.get("period_of_report")
    metadata["state_location"] = record.get("state_location")
    metadata["fiscal_year_end"] = record.get("fiscal_year_end")
    metadata["htm_filing_link"] = record.get("htm_filing_link")
    metadata["filename"] = record.get("filename")

    return metadata

In [8]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("reports_l2")

In [9]:
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)

# This text splitter is used to create the parent documents - The big chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# This text splitter is used to create the child documents - The small chunks
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(client = persistent_client,
                     collection_name="reports_l2",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb')


In [10]:
big_chunks_retriever = ParentDocumentRetriever(
    # The underlying vectorstore to use to store small chunks and their embedding vectors
    vectorstore=vectorstore,
    # The storage interface for the parent documents
    docstore=store,
    # The text splitter to use to create child documents.
    child_splitter=child_splitter,
    # The text splitter to use to create parent documents.
    parent_splitter=parent_splitter,
    #search_kwargs={'k': 2, 'search_type': 'similarity'}
)

# by default the search_type is 'similarity, also 'mmr' and 'similarity_score_threshold' are available

In [11]:
import os

directory = "/content/drive/MyDrive/Thesis/FinanceBench"
total_number = 0
for filename in os.listdir(directory):
  if filename.endswith('.json'):
    file_path = os.path.join(directory, filename)
    loader = JSONLoader(file_path=file_path, jq_schema='.', content_key='content', metadata_func=metadata_function)
    data = loader.load()
    big_chunks_retriever.add_documents(data)
    total_number += 1
    print(f'The number of documents added: {total_number}')
    print(f'The last added document is: {filename}')
    print(f'The number of parent docs is: {len(list(store.yield_keys()))}')

The number of documents added: 1
The last added document is: 1410636_10K_2020_0001410636-21-000101.json
The number of parent docs is: 357
The number of documents added: 2
The last added document is: 1018724_10K_2017_0001018724-18-000005.json
The number of parent docs is: 520
The number of documents added: 3
The last added document is: 1018724_10K_2019_0001018724-20-000004.json
The number of parent docs is: 674
The number of documents added: 4
The last added document is: 1748790_10Q_2023_0001748790-23-000021.json
The number of parent docs is: 742
The number of documents added: 5
The last added document is: 104169_10K_2018_0000104169-18-000028.json
The number of parent docs is: 945
The number of documents added: 6
The last added document is: 1403568_10K_2023_0001558370-23-004581.json
The number of parent docs is: 1098
The number of documents added: 7
The last added document is: 1637459_10K_2019_0001637459-20-000027.json
The number of parent docs is: 1390
The number of documents added: 8


In [12]:
vectorstore.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [13]:
all = vectorstore.get()['metadatas']
doc_ids = []
ciks = []
for i in range(len(all)):
  doc_ids.append(all[i]['doc_id'])
  ciks.append(all[i]['cik'])

print(f"The number of unique companies: {len(set(ciks))}")
print(f"The number of parent documents: {len(set(doc_ids))}")
print(f"The number of child documnets: {len(vectorstore.get()['documents'])}")

The number of unique companies: 32
The number of parent documents: 17561
The number of child documnets: 356275


# Cloning collections with different distance metrics

In [14]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("reports_l2")  # distance function L2 by default

newCollection = persistent_client.get_or_create_collection("reports_cosine", metadata={"hnsw:space": "cosine"})  # distance function - cosine

existing_count = collection.count()
batch_size = 5000
for i in range(0, existing_count, batch_size):
    batch = collection.get(include=["metadatas", "documents", "embeddings"], limit=batch_size, offset=i)
    newCollection.add(ids=batch["ids"], documents=batch["documents"], metadatas=batch["metadatas"],
               embeddings=batch["embeddings"])

print(newCollection.count()) #356275

356275


In [15]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("reports_l2")  # distance function L2 by default

newCollection = persistent_client.get_or_create_collection("reports_ip", metadata={"hnsw:space": "ip"})  # distance function - inner product

existing_count = collection.count()
batch_size = 10000
for i in range(0, existing_count, batch_size):
    batch = collection.get(include=["metadatas", "documents", "embeddings"], limit=batch_size, offset=i)
    newCollection.add(ids=batch["ids"], documents=batch["documents"], metadatas=batch["metadatas"],
               embeddings=batch["embeddings"])

print(newCollection.count()) #356275

356275
